# Lab assigment 5
## Bloom filters

### Pedro Otero García & Alexandre Sousa Cajide

### Step 1
#### Set-up the bloom filter

In [9]:
import mmh3

class BloomFilter:
  def __init__(self, m, k, q):
      self.m = m  # Size of the filter
      self.k = k  # Number of hash functions
      self.q = q  # q-grams

      # Create a bit array of size m, initialized with zeros
      # This are going to be the bloom filter
      self.filter = [0] * m

  def add(self, name: str):
    for i in range(len(name) - self.q + 1):
      self._addQgram(name[i:i+self.q])

  def _addQgram(self, qgram):
    # Double hash q-gram and set corresponding bits to 1
    for i in range(1, self.k + 1):
        hash1 = mmh3.hash(qgram)
        hash2 = mmh3.hash(qgram, 1)
        index = (hash1 + i * hash2) % self.m
        self.filter[index] = 1

  def check(self, name: str):
      if len(name) % 2 != 0:
        name += "_"
      for i in range(len(name) - self.q + 1):
        if not self._checkQgram(name[i:i+self.q]):
          return False
      return True

  def _checkQgram(self, qgram):
      # Check if all bits are set to 1 for the given q-gram
      for i in range(1, self.k + 1):
          hash1 = mmh3.hash(qgram)
          hash2 = mmh3.hash(qgram, 1)
          index = (hash1 + i * hash2) % self.m
          if self.filter[index] == 0:
              return False
      return True


# Define parameters
m = 10000  # Size of the filter
k = 5     # Number of hash functions
q = 2     # Number of q-grams

# Create and initialize the Bloom Filter
bloom_filter = BloomFilter(m, k, q)
bloom_filter.add("Alex")
bloom_filter.add("Pedro")

# Check if a q-gram is in the filter
print("Alex:",bloom_filter.check("Alex"))
print("Anthea:",bloom_filter.check("Anthea"))

Alex: True
Anthea: False


### Step 2

#### Reading datasets.

In [10]:
import pandas as pd
import numpy as np

# - Dataset example names
en = pd.read_csv('pprl-attack-data/example-names.csv')
print(en.head(5))

# - German names
gn = pd.read_csv('pprl-attack-data/german-names.csv')
print(gn.head(5))

     name  frequency
0  Daniel        242
1  Carlos        130
2  Danilo        115
3   Carla         48
4   David         95
       name  frequency
0     Peter    1293922
1   Michael    1088118
2  Wolfgang    1075005
3    Thomas    1011277
4     Klaus     992990


#### Generate a realistic data.

In [11]:
import random

def genDataFrame(df, s):
  # - Generate data frame of length l using the names
  #   in df with its corresponding f.

  samples = []

  for _, row in df.iterrows():
      name = row['name']
      f = row['frequency']

      samples.extend([name] * f)

  data = np.random.choice(samples, size=s, replace=True)

  df_generated = pd.DataFrame({'name': data})

  return df_generated

def selectNnames(df,n):
  # - n random rows of a dataframe.
  sampled_df = df.sample(n=n, random_state=random.seed())

  return sampled_df


df = selectNnames(gn,n=5)
print(list(df['name'].unique()))
df = genDataFrame(df,1000)
print(list(df['name'].unique()))

['Tim', 'Christl', 'Susan', 'Romy', 'Frida']
['Tim', 'Susan', 'Christl', 'Frida', 'Romy']


#### Generate a realistic scenario with optimal parameters.

In [24]:
import math

def genScenario(n: int, m: int, k: int, s: int) -> [pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        '''
            Generate a real scenario.
            @params:
                n: Number of different names in the synthetic dataset.
                m: Length of bloom filters. 
                k: Number of hash functions.
                s: Length of the synthetic dataset

            @returns:
                df: Synthetic dataset of names.
                bfs: Bloom filters dataset for each row in df.
                real_df: Dataset with the names in df and its corresponding bloom filter.
        '''
    
        df = selectNnames(gn,n)
        df = genDataFrame(df,s)

        filters = []
        freqs = []
        for name in df['name'].unique():
                bf = BloomFilter(m, k, q)
                bf.add(name)
                filters.append(bf.filter)
                freqs.append(df['name'].value_counts()[name])
        bfs = pd.DataFrame({'filter': filters, 'frequency': freqs})

        # 'real_df' are going to be used to check the result
        # of the attack.
        real_df = pd.DataFrame()
        real_df['name'] = df['name'].unique()
        real_df['filter'] = bfs['filter']

        return df, bfs, real_df


# - Parameters selected by the user.
n = 500
# - Optimal parameters.
epsilon = 5e-8
m = round(-n*math.log(epsilon)/(math.log(2) ** 2))
k = round(m/n*math.log(2))

print('Epsilon =',epsilon)
print('n =',n)
print('m =',m)
print('k =',k)
print()

df, bfs, real_df = genScenario(n=n,m=m,k=k,s=10000)
print(df.head())
print()
print(real_df.head())

Epsilon = 5e-08
n = 500
m = 17495
k = 24

       name
0     Gunda
1    Martin
2   Barbara
3  Heribert
4    Sabine

       name                                             filter
0     Gunda  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    Martin  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2   Barbara  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3  Heribert  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    Sabine  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...


### Step 3

#### Code of the attacker

In [83]:
def computeQgrams(name: str, q: int) -> list:
  q_grams = []
  for i in range(len(name) - q + 1):
        q_grams.append(name[i:i+q])

  return q_grams

def getPos1(filter: list) -> list:
    pos1 = []
    for pos,i in enumerate(filter):
        if i == 1:
            pos1.append(pos)
    return pos1


def attack(q: int, t: int, df: pd.DataFrame, bfs: pd.DataFrame) -> [list,list,list]:
    '''
        Re-identification attack.
        @params:
            q: number of q-grams.
            t: frequency threshold.
            df: df to perform the re-identification.
            bfs: dataset of the bloom filters.

        @returns:
            cand_qgs: Candidates q-grams per position of bloom filter.
            cand_nms: Candidates names per bloom filter.
            list[bfs['filter']]: Bloom filters.
    '''

    print("Bloom filters:")
    bfs = bfs[bfs['frequency'] >= t]
    bfs = bfs.sort_values(by='frequency', ascending=False)
    print(bfs.head(5))
    print("Length:",len(bfs))
    print()

    print("Dataframe for the attack:")
    df4attack = df['name'].value_counts().reset_index()
    df4attack.columns = ['name', 'frequency']
    df4attack = df4attack[df4attack['frequency'] >= t]
    
    qgrams = []
    for name in df4attack['name']:
        qgrams.append(computeQgrams(name, q))

    df4attack['q-grams'] = qgrams
    df4attack = df4attack.sort_values(by='frequency', ascending=False)
    print(df4attack.head(5))
    print("Length:",len(df4attack))

    # Candidates q-grams
    cand_qgs = []

    # Computing candidates for each position in filter's length
    for i in range(0,len(bfs['filter'].iloc[0])):
        pos_cands = [] # Possitive candidates set.
        neg_cands = [] # Negative candidates set.
        for j, flt in enumerate(bfs['filter']):
            qgs = df4attack['q-grams'].iloc[j]
            if flt[i] == 1:
                pos_cands = list(set(pos_cands + qgs))
            else:
                neg_cands = list(set(neg_cands + qgs))
        cand_qgs.append(list(set(pos_cands)-set(neg_cands)))
        
    # Re-identification
    cand_nms = []
    for flt in bfs['filter']:
        cn = list(df4attack['name'])
        # Checking each position where filter is equal to '1'.
        for pos in getPos1(flt):
            for _, row in df4attack[['name', 'q-grams']].iterrows():
                # Checking if there ARE NOT candidates for a name
                # in position 'pos'.
                if not any(qg in cand_qgs[pos] for qg in row['q-grams']):
                    try:
                        cn.remove(row['name'])
                    except:
                        continue
        cand_nms.append(cn)

        
    return cand_qgs, cand_nms, list(bfs['filter'])

### Step 4

#### Performing the attack.

In [85]:
cand_qgs, cand_nms, filters = attack(q=2, t=50, df=df, bfs=bfs)

Bloom filters:
                                               filter  frequency
59  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        257
12  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        232
46  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        228
26  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        217
33  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        215
Length: 58

Dataframe for the attack:
       name  frequency                       q-grams
0     Peter        257              [Pe, et, te, er]
1   Michael        232      [Mi, ic, ch, ha, ae, el]
2  Wolfgang        228  [Wo, ol, lf, fg, ga, an, ng]
3     Klaus        217              [Kl, la, au, us]
4   Manfred        215      [Ma, an, nf, fr, re, ed]
Length: 58


#### Analyzing results:

In [106]:
print("Total Bloom filters to identify:", len(cand_nms))
print(cand_nms)
for i, cnd in enumerate(cand_nms):
    print((real_df[real_df['name']==cnd[0]]['filter'].iloc[0]) == filters[i])
    

Total Bloom filters to identify: 58
[['Peter'], ['Michael'], ['Wolfgang'], ['Klaus'], ['Manfred'], ['Hans'], ['Werner'], ['Maria', 'Marianne'], ['Helmut'], ['Sabine'], ['Dieter'], ['Helga'], ['Günter'], ['Ingrid'], ['Petra'], ['Gisela'], ['Josef'], ['Claudia'], ['Frank'], ['Christa'], ['Stefan'], ['Martin', 'Martina'], ['Barbara'], ['Marianne'], ['Georg'], ['Karl-Heinz'], ['Birgit'], ['Inge', 'Ingeborg'], ['Martina'], ['Susanne'], ['Ute'], ['Rudolf'], ['Norbert'], ['Anna'], ['Rainer'], ['Alfred'], ['Waltraud'], ['Markus'], ['Matthias'], ['Ulrike'], ['Nicole'], ['Willi'], ['Hannelore'], ['Erich'], ['Ingeborg'], ['Anneliese'], ['Regina'], ['Ralf'], ['Dirk'], ['Ilse'], ['Reinhard'], ['Günther'], ['Silke'], ['Erwin'], ['Eva'], ['Sandra'], ['Sonja'], ['Anja']]
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True

In [93]:
a = [0,1,0,0,1]
b = [0,1,0,1,1]
a == b

False